# __Naive Bayes__

In [2]:
import scipy.io #Bibliotecas de Utilização no projeto
import numpy as np
import pandas as pd
data=scipy.io.loadmat('data.mat')

## __Representação da Matriz de Dados de Treinamento__

In [3]:
dados_treinamento=pd.DataFrame(data['X'])
dados_treinamento['Y']=data['Y'].T
dados_treinamento.index=range(1,1001,1)#mudar o range posição inteiras
#Nomenclatura da Matrix em função das palavras Ws
dados_treinamento.columns=pd.Index(['W1','W2','W3','W4','W5','Y'], dtype='object')
dados_treinamento

,W1,W2,W3,W4,W5,Y
1,8,4,4,1,3,-1
2,2,12,0,3,3,1
3,8,2,7,1,2,-1
4,5,1,9,1,4,-1
5,8,3,5,1,3,-1
6,6,4,6,1,3,-1
7,10,0,6,0,4,-1
8,12,1,4,1,2,-1
9,9,2,4,1,4,-1
10,6,5,1,2,6,1


# __Cálculo das Probabilidades Iniciais__ 

In [5]:
tamanho=len(dados_treinamento)
print(tamanho)
py1=len(dados_treinamento[dados_treinamento['Y']==1])/tamanho
py0=len(dados_treinamento[dados_treinamento['Y']==-1])/tamanho
print(py1,py0)

1000
0.506 0.494


In [6]:
#X=dados_treinamento.iloc[:,:-1] #Filtrando primeiras colunas

#dados de treinamento com rótulo -
xyneg=dados_treinamento[dados_treinamento['Y']==-1]
#dados de treinamento com rótulo +
xypos=dados_treinamento[dados_treinamento['Y']==1]

npos=xypos.iloc[:,:-1].to_numpy().sum() #soma de todas as palavras onde y é +1
nneg=xyneg.iloc[:,:-1].to_numpy().sum() #soma de todas as palavras onde y é -1

print(npos,nneg)

10120 9880


Pela definição de LaPlace Smothing temos que para determinada palavra:


$$p(w_k|y=1)=\frac{n_k+1}{n+|V|}$$


Assim como para o caso negativo

$$p(w_k|y=-1)=\frac{n_k+1}{n+|V|}$$


In [7]:
opws=['w1','w2','w3','w4','w5']#ocorrencias positivas das palavras
onws=['w1','w2','w3','w4','w5']##..neg ..

soma_colunas_ypos=list(xypos.iloc[:,:-1].sum().to_numpy())
soma_colunas_yneg=list(xyneg.iloc[:,:-1].sum().to_numpy())

opws=dict(zip(opws, soma_colunas_ypos))
onws=dict(zip(onws, soma_colunas_yneg))
print(opws,onws)

pwkypos,pwkyneg={},{}#probabilidades ocorrencia pos
#pwkyneg=['w1', 'w2','w3','w4','w5']#probabilidades ocorrencia neg

#funcao curta de L.Smoothing, recebe nk(-numero de ocor da palavra) e n(soma palavras)
pwk=lambda nk,n: (nk+1)/(n+5)#onde 5 é o n° de features, palavras ou classes

for key in opws.keys():
    pwkypos[key]=pwk(opws[key],npos)

for key in onws.keys():
    pwkyneg[key]=pwk(onws[key],nneg)

print(pwkypos,pwkyneg)    

{'w1': 2003, 'w2': 3035, 'w3': 1076, 'w4': 1530, 'w5': 2476} {'w1': 3910, 'w2': 953, 'w3': 2981, 'w4': 523, 'w5': 1513}
{'w1': 0.19792592592592592, 'w2': 0.29985185185185187, 'w3': 0.10637037037037037, 'w4': 0.1512098765432099, 'w5': 0.24464197530864198} {'w1': 0.3956499747091553, 'w2': 0.09650986342943854, 'w3': 0.3016691957511381, 'w4': 0.0530096105209914, 'w5': 0.15316135558927668}


"pwkypos" e "pwkyneg" são respectivamente as probabilidades a priori de cadas uma das palavras, optei por salva-las em uma estrutura de dados de dicionário, evidenciando cada uma delas indivualmente. Podemos mapea-las através de um Dataframe com a seguinte representação:

In [8]:
probs=pd.DataFrame(index=range(1,6))
probs['Prob Palavra i dado y +']=pwkypos.values()
probs['Prob Palavra i dado y -']=pwkyneg.values()


probs

,Prob Palavra i dado y +,Prob Palavra i dado y -
1,0.197926,0.395650
2,0.299852,0.096510
3,0.106370,0.301669
4,0.151210,0.053010
5,0.244642,0.153161


# __Funcionamento do classificador__

In [49]:
teste=[]
teste=dados_treinamento.copy()

x_train=teste.iloc[:,:-1].to_numpy()

def classificacao(xdados,df,dp1=[],dp2=[],rotulo=[]):
    for i in range(len(xdados)):
        l0=xdados[i,:]
        ps=np.array(list(pwkypos.values()))
        ns=np.array(list(pwkyneg.values()))
        tempp=float(np.dot(l0,ps))*py1
        tempn=float(np.dot(l0,ns))*py0
        dp1.append(tempp)
        dp2.append(tempn)
        
        #checagem do maior produtório das probs abaixo
        if tempp>tempn:
            rotulo.append(1)
        else:
            rotulo.append(-1)
        
    #inserção de novos campos no dataframe
    print(len(dp1))
    df['dp1']=dp1
    df['dp2']=dp2
    df['Predição']=rotulo
        
classificacao(x_train,teste)        

#Classifica corretamente para todas os dados de treinamento
print(len(teste['Predição'].to_numpy()==teste['Y'].to_numpy()))

teste.head(10)#Primeiros dados

1000
1000


,W1,W2,W3,W4,W5,Y,dp1,dp2,Predição
1,8,4,4,1,3,-1,2.071277,2.603582,-1
2,2,12,0,3,3,1,2.621905,1.268558,1
3,8,2,7,1,2,-1,1.805508,2.879643,-1
4,5,1,9,1,4,-1,1.708556,2.694986,-1
5,8,3,5,1,3,-1,1.973375,2.704931,-1
6,6,4,6,1,3,-1,1.978622,2.510729,-1
7,10,0,6,0,4,-1,1.819601,3.151305,-1
8,12,1,4,1,2,-1,1.892915,3.166697,-1
9,9,2,4,1,4,-1,1.991766,2.779343,-1
10,6,5,1,2,6,1,2.309109,2.066454,1


# __Implementação da Classificação para a nova matriz de dados__

In [50]:
dados_teste=pd.DataFrame(data['newX'])

x_array=dados_teste.to_numpy()#Matriz de características
print(len(x_array), len(dados_teste))
#fazendo alguns arranjos
dados_teste.index=range(1,501,1)
dados_teste.columns=pd.Index(['W1','W2','W3','W4','W5'], dtype='object')

#Coluna com valores reais
classificacao(x_array,dados_teste,[],[],[])#passar atributos de lista vazia

dados_teste['newY']=data['newY'].T

dados_teste.tail(25)#Últimos dados

500 500
500


,W1,W2,W3,W4,W5,dp1,dp2,Predição,newY
476,5,3,1,4,7,2.182322,1.903687,1,1
477,2,6,1,8,3,2.147939,1.262461,1,1
478,8,6,2,1,3,2.267080,2.400885,-1,1
479,7,2,8,1,2,1.759181,2.833216,-1,-1
480,4,6,0,4,6,2.359734,1.626577,1,1
481,6,6,2,1,5,2.314357,2.161306,1,1
482,5,4,4,3,4,2.047638,2.145264,-1,1
483,10,4,4,1,1,2.024000,2.843161,-1,-1
484,9,0,9,0,2,1.633343,3.251604,-1,-1
485,9,2,4,1,4,1.991766,2.779343,-1,-1


Foi possível observar que ocorrem algumas divergência entre o valor de predição e o valor real da classificação, visualizando os últimos 25 dados do problema. O endereço 482 e 478 apresentaram falhas na predição.
O percentual de classificação correto pode ser definido como:

In [53]:
len(dados_teste[dados_teste['newY']==dados_teste['Predição']])

472

Ou seja, foram classificados corretamente 472 dados de 500. O que nos dá uma percentual de classificação de:

In [54]:
472/500

0.944

Confiança de classificação da ordem de $94.4$%

# Exportando dados

In [55]:
dados_teste.to_excel('dados_teste.xlsx') #Arquivo de comparação das classificação com novos dados testados
teste.to_excel('analise_treinamento.xlsx') #Arquivo dos modelos com o set de treinamento